In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter

df = pd.read_csv(r'C:\Users\TRITON300SE\Desktop\Machine Learning\Ceub\Prova\Guilherme\Scrapping Apt Brasília Venda 1000.csv')
df.head()

,Unnamed: 0,Link,Preco,Localizacao,Area_Util_m2,Quartos,Suites,Vagas_Garagem
0,0,https://www.dfimoveis.com.br/imovel/lancamento...,4510049.0,BRASÍLIA - NOROESTE,238,4.0,4.0,2.0
1,1,https://www.dfimoveis.com.br/imovel/apartament...,1400000.0,BRASÍLIA - ASA NORTE,141,2.0,NaN,2.0
2,2,https://www.dfimoveis.com.br/imovel/apartament...,1529000.0,BRASÍLIA - SUDOESTE,79,2.0,2.0,1.0
3,3,https://www.dfimoveis.com.br/imovel/apartament...,1395000.0,BRASÍLIA - NOROESTE,85,2.0,2.0,2.0
4,4,https://www.dfimoveis.com.br/imovel/lancamento...,NaN,BRASÍLIA - NOROESTE,165,4.0,4.0,3.0


In [3]:
df = df[['Localizacao', 'Area_Util_m2', 'Quartos', 'Suites', 'Vagas_Garagem', 'Preco']]

df = df.rename(columns={
    'Localizacao' : 'localizacao',
    'Area_Util_m2' : 'area_util',
    'Quartos' : 'quartos',
    'Suites' : 'suites',
    'Vagas_Garagem' : 'qntd_vagas',
    'Preco' : 'preco'
})

print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   localizacao  1000 non-null   object 
 1   area_util    1000 non-null   int64  
 2   quartos      999 non-null    float64
 3   suites       805 non-null    float64
 4   qntd_vagas   816 non-null    float64
 5   preco        964 non-null    float64
dtypes: float64(4), int64(1), object(1)
memory usage: 47.0+ KB
None
         area_util     quartos      suites  qntd_vagas         preco
count  1000.000000  999.000000  805.000000  816.000000  9.640000e+02
mean    136.958000    2.762763    1.924224    2.011029  2.028010e+06
std      95.084148    1.040644    1.178359    1.130123  1.695783e+06
min       0.000000    1.000000    1.000000    0.000000  1.800000e+05
25%      72.000000    2.000000    1.000000    1.000000  9.515472e+05
50%     120.000000    3.000000    1.000000    2.000000  1.650000e+06
75%     

In [4]:
print('Valores Nulos a Serem removidos...')
print(df.isnull().sum())

print('\nApós a remocao de nulos em area_util e preco')
df_limpo = df.dropna(subset=['area_util', 'preco'])
print(df_limpo.isnull().sum())


print('\nSubstituindo os valores nulos restantes pela mediana')
colunas = ['quartos', 'suites', 'qntd_vagas']
df_limpo = df_limpo.copy()
for i in colunas:
    mediana_agrupada = df_limpo.groupby('localizacao')[i].transform('median')
    df_limpo[i] = df_limpo[i].fillna(mediana_agrupada)

if df_limpo.isnull().sum().sum() > 0:

    medianas_globais = df_limpo.median(numeric_only=True)
    df_limpo.fillna(medianas_globais, inplace=True)

print(df_limpo.isnull().sum())

Valores Nulos a Serem removidos...
localizacao      0
area_util        0
quartos          1
suites         195
qntd_vagas     184
preco           36
dtype: int64

Após a remocao de nulos em area_util e preco
localizacao      0
area_util        0
quartos          1
suites         194
qntd_vagas     183
preco            0
dtype: int64

Substituindo os valores nulos restantes pela mediana
localizacao    0
area_util      0
quartos        0
suites         0
qntd_vagas     0
preco          0
dtype: int64


In [5]:
grupos_por_localizacao = df_limpo.groupby('localizacao')
colunas_para_analise = ['preco', 'area_util', 'quartos', 'suites', 'qntd_vagas']


print("\n--- Limites de Outlier Calculados para Cada Localização ---")

for nome_localizacao, grupo_df in grupos_por_localizacao:
    print(f"\n=======================================================")
    print(f"Localização: {nome_localizacao} ({len(grupo_df)} imóveis)")
    print(f"=======================================================")
    
    for coluna in colunas_para_analise:
        if grupo_df[coluna].count() < 2:
            print(f"  Coluna: '{coluna}' -> Dados insuficientes para calcular limites.")
            continue

        Q1 = grupo_df[coluna].quantile(0.25)
        Q3 = grupo_df[coluna].quantile(0.75)
        IQR = Q3 - Q1
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR

        print(f"  Coluna: '{coluna}'")
        print(f"    - Limite Inferior: {limite_inferior:,.2f}")
        print(f"    - Limite Superior: {limite_superior:,.2f}")


--- Limites de Outlier Calculados para Cada Localização ---

Localização: BRASÍLIA - ASA NORTE (263 imóveis)
  Coluna: 'preco'
    - Limite Inferior: -697,500.00
    - Limite Superior: 3,322,500.00
  Coluna: 'area_util'
    - Limite Inferior: -25.25
    - Limite Superior: 232.75
  Coluna: 'quartos'
    - Limite Inferior: 0.50
    - Limite Superior: 4.50
  Coluna: 'suites'
    - Limite Inferior: 1.00
    - Limite Superior: 1.00
  Coluna: 'qntd_vagas'
    - Limite Inferior: 1.00
    - Limite Superior: 1.00

Localização: BRASÍLIA - ASA SUL (152 imóveis)
  Coluna: 'preco'
    - Limite Inferior: -625,000.00
    - Limite Superior: 3,975,000.00
  Coluna: 'area_util'
    - Limite Inferior: -21.12
    - Limite Superior: 263.88
  Coluna: 'quartos'
    - Limite Inferior: 3.00
    - Limite Superior: 3.00
  Coluna: 'suites'
    - Limite Inferior: 1.00
    - Limite Superior: 1.00
  Coluna: 'qntd_vagas'
    - Limite Inferior: 1.00
    - Limite Superior: 1.00

Localização: BRASÍLIA - JARDIM BOTANICO 

In [6]:
df_limpo.to_csv(r'df_limpo.csv', index=False)

<H3> Conclusão: Após limpeza dos dados e insperção dos ouliers, foi visto que não é necessários removê-los pois representam valores totalmente possíveis. Provavelmente a plataforma DF Imóveis tem um bom controle em relação a inserção de dados em seus anúncios. 